In [130]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Forward and Backwards Passes

In [131]:
#export 
from exp.nb_01 import * 

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f: 
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
#NB: Use Training not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(-6.2598e-06), tensor(1.))

In [11]:
#export 
def test_near_zero(a, tol=1e-3): assert a.abs()<tol, f"Near Zero: {a}"

In [14]:
test_near_zero(x_train.mean())
test_near_zero(x_train.std()-1)

In [15]:
n,m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

## Foundations Version

### Basic Structure 

In [16]:
#num hidden
nh = 50 

In [71]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [72]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [73]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [74]:
def lin(x, w, b): return x@w + b

In [75]:
t = lin(x_valid, w1, b1)

In [76]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0391), tensor(0.9506))

In [77]:
def relu(x): return x.clamp_min(0.)

In [78]:
t = relu(lin(x_valid, w1, b1))

In [79]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3963), tensor(0.5770))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

In [80]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [81]:
w1.mean(),w1.std()

(tensor(-7.2285e-06), tensor(0.0508))

In [82]:
t = relu(lin(x_valid, w1, b1))

In [83]:
t.mean(), t.std()

(tensor(0.5411), tensor(0.7933))

In [84]:
#export 
from torch.nn import init

In [85]:
w1 = torch.zeros(m, nh)
init.kaiming_normal_(w1, mode='fan_out') #means preserves the magintudes of backward pass
t = relu(lin(x_valid, w1, b1))

In [89]:
# init.kaiming_normal_??

In [90]:
w1.mean(), w1.std()

(tensor(-0.0002), tensor(0.0501))

In [91]:
t.mean(), t.std()

(tensor(0.6170), tensor(0.8885))

In [92]:
w1.shape

torch.Size([784, 50])

In [93]:
import torch.nn

In [94]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

So realize that pytorch flips things around as you can see so fan_in is doing the same thing fan_out is doing for what we have going on.

In [ ]:
# torch.nn.Linear.forward??

In [ ]:
# torch.nn.functional.linear??

In [49]:
# torch.nn.Conv2d??

In [95]:
# torch.nn.modules.conv._ConvNd.reset_parameters??

In [96]:
def relu(x): return x.clamp_min(0.) - 0.5

In [97]:
w1 = torch.randn(m, nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
#we want 0 and 1
t1.mean(), t1.std()

(tensor(0.0833), tensor(0.8216))

In [98]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [99]:
%timeit -n 10 _=model(x_valid)

4.26 ms ± 1.67 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Loss Function: MSE

In [100]:
model(x_valid).shape

torch.Size([10000, 1])

We need ``squeeze()`` to get rid of the trailing (,1) in order to use ``mse``. (Of course, ``mse`` is not a suitable loss function for a multi-class classification problem; we'll use a better loss function soon.)

In [101]:
#export 
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [102]:
y_train, y_train = y_train.float(), y_train.float()

In [105]:
preds = model(x_train)

In [106]:
preds.shape

torch.Size([50000, 1])

In [107]:
mse(preds, y_train)

tensor(26.1007)

### Gradients and Backward Pass

In [171]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [172]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [173]:
def lin_grad(inp, out, w, b):
    #grad of matmul with respect to input 
    inp.g = out.g @ w.t()
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [174]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [175]:
forward_and_backward(x_train, y_train)

In [176]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

In [177]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [178]:
def forward(inp, targ): 
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    return mse(out, targ)

In [179]:
loss = forward(xt2, y_train)

In [180]:
loss.backward()

In [181]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor Model

### Layers as classes

In [196]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [197]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [198]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [199]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [200]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [201]:
%time loss = model(x_train, y_train)

CPU times: user 132 ms, sys: 12.9 ms, total: 145 ms
Wall time: 24.4 ms


In [202]:
%time model.backward()

CPU times: user 7.69 s, sys: 3.02 s, total: 10.7 s
Wall time: 1.78 s


In [203]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

###  Module.forward()

In [210]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('Not Implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [211]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [212]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [213]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [214]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [215]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [216]:
%time loss = model(x_train, y_train)

CPU times: user 142 ms, sys: 22.6 ms, total: 165 ms
Wall time: 27.4 ms


In [217]:
%time model.backward()

CPU times: user 443 ms, sys: 240 ms, total: 683 ms
Wall time: 124 ms


In [218]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### With out einsum

In [219]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [220]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [221]:
%time loss = model(x_train, y_train)

CPU times: user 158 ms, sys: 17.4 ms, total: 175 ms
Wall time: 29.2 ms


In [222]:
%time model.backward()

CPU times: user 364 ms, sys: 97 ms, total: 461 ms
Wall time: 76.8 ms


In [223]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [224]:
#export 
from torch import nn

In [235]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [236]:
model = Model(m, nh, 1)

In [237]:
%time loss = model(x_train, y_train)

CPU times: user 152 ms, sys: 1.93 ms, total: 154 ms
Wall time: 53.9 ms


In [238]:
%time loss.backward()

CPU times: user 150 ms, sys: 54.4 ms, total: 204 ms
Wall time: 34.3 ms


## Export

In [241]:
!  python notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py


In [242]:
!cat ./exp/nb_02.py


#################################################
### THIS FILE WAS AUTOGENERATED! DO NOT EDIT! ###
#################################################
# file to edit: dev_nb/02_fully_connected.ipynb
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, m, s): return (x-m)/s

def test_near_zero(a, tol=1e-3): assert a.abs()<tol, f"Near Zero: {a}"

from torch.nn import init

def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

from torch import nn